In [1]:
import pandas as pd
import os
from tqdm import tqdm

from glob import glob
from commons import plot, check_colision_border

from skimage.io import imread_collection, imread, imsave
from skimage.measure import label as region_label, regionprops_table, regionprops

from skimage import (
    img_as_ubyte,
)

In [2]:
def extrat_area(clusters):
    
    return max(map(lambda item: item.area, regionprops(clusters)))


def load_paths_images_sorted(path):

    arr = []

    def parser_image_name(image_name):

        *_, name = image_name.split("/")
        name, *_ = name.split(".")

        return int(name)

    for index in tqdm(glob(f'{path}/*')):

        try:
            image_name = parser_image_name(fix_path(index))
    
            arr.append(image_name)

        except: continue

    image_path_sorted = sorted(arr)

    image_unique_name = lambda x: f"{path}/{x}.tif"

    return list(map(image_unique_name, image_path_sorted))

def select_only_valid_masks(image_paths):
    images = []
    paths = []
    
    for (img, path) in zip(imread_collection(image_paths), image_paths):
        if not check_colision_border(img):
            images.append(img)
            paths.append(path)
        
    return {
        'images': images,
        'paths': paths
    }

In [3]:
def fix_path(path):
    return path.replace("\\", "/")

In [4]:
_input = 'many'

In [5]:
# os.mkdir('auto_selected_masks')

for index_image in glob(f'{_input}/*'):
    
    image_paths = load_paths_images_sorted(index_image)
    images_paths = select_only_valid_masks(image_paths)
    
    images_area_pd = pd.DataFrame({
        'area': map(extrat_area, images_paths['images']),
        'path': images_paths['paths']
    })

    areas = images_area_pd.area.to_list()
    median = areas[len(areas) // 2] 
    
    query = images_area_pd.area == median
    best_mask, *_ = images_area_pd[query].path.to_list()
    
    image_name = fix_path(index_image).split("/")[1]
    imsave(f'auto_selected_masks/{image_name}.tif', img_as_ubyte(imread(best_mask)))

100%|██████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<?, ?it/s]
